In [1]:
import albumentations as A
from PIL import Image
import os
import numpy as np
import cv2
from tqdm import tqdm

In [2]:
IMAGE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier_Compact\Datasets\Cover_Images"
AUGMENT_SAVE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier_Compact\Datasets\Augment"
AUGMENT_COUNT = 500
N = 256

In [3]:
transform = A.Compose([
    A.LongestMaxSize(max_size=N, interpolation=1),
    A.PadIfNeeded(min_height=N, min_width=N, border_mode=0, value=(0,0,0)),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=45, p=0.75, border_mode=cv2.BORDER_CONSTANT),
    A.OneOf([
        A.HorizontalFlip(p=1),
        A.VerticalFlip(p=1)
    ], p=1),
    A.OneOf([
        A.Blur(p=1),
        A.OpticalDistortion(p=1),
        A.GaussNoise(p=1)
    ], p=1)
])

In [4]:
for folder in tqdm(os.listdir(IMAGE_PATH)):
    count = 0
    for image_name in os.listdir(f"{IMAGE_PATH}\{folder}"):
        image = Image.open(f"{IMAGE_PATH}\{folder}\{image_name}")
        image = np.array(image)

        SAVE = f"{AUGMENT_SAVE_PATH}\{folder}"

        try: os.mkdir(SAVE)
        except FileExistsError: pass
        
        try:
            for i in range(AUGMENT_COUNT):
                aug = transform(image=image)
                aug_img = Image.fromarray(aug["image"])
                aug_img.save(f"{SAVE}\img{count}-{i}.png")
        except:
            continue
        
        count += 1

100%|███████████████████████████████████████████████████████████████████████████████| 12/12 [1:27:23<00:00, 436.99s/it]


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [8]:
IMAGE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier_Compact\Datasets\Augment"
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32
NUM_CLASSES = 12

train = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
train = train.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

validation = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
validation = validation.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

train = train.prefetch(buffer_size=32)
validation = validation.prefetch(buffer_size=32)

Found 338500 files belonging to 12 classes.
Using 270800 files for training.
Found 338500 files belonging to 12 classes.
Using 67700 files for validation.


In [9]:
import h5py
import os, shutil
import matplotlib.pyplot as plt
from keras.optimizers import SGD
from keras import layers, models, optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from squeezenet import SqueezeNet
from keras.callbacks import EarlyStopping

In [10]:
sn = SqueezeNet(input_shape=(256, 256, 3), nb_classes=NUM_CLASSES)
sgd = SGD(lr=0.001, decay=0.0002, momentum=0.9, nesterov=True)
sn.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

CHECKPOINT_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier_Compact\Models\Keras SqueezeNet Model\h5 Model\weights.h5"
EPOCHS = 300

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=0)
checkpoint = ModelCheckpoint(                                         
    CHECKPOINT_PATH,
    monitor='val_loss',                               
    verbose=0,                                        
    save_best_only=True,                              
    save_weights_only=True,                           
    mode='min',                                       
    period=1
)

In [11]:
sn.fit_generator(
    train,
    validation_data=validation,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[checkpoint]
)

Epoch 1/300
8463/8463 [==============================] - 289s 33ms/step - loss: 1.8905 - accuracy: 0.3402 - val_loss: 1.2461 - val_accuracy: 0.5736
Epoch 2/300
8463/8463 [==============================] - 261s 31ms/step - loss: 0.4975 - accuracy: 0.8350 - val_loss: 0.1220 - val_accuracy: 0.9602
Epoch 3/300
8463/8463 [==============================] - 260s 31ms/step - loss: 0.0854 - accuracy: 0.9743 - val_loss: 0.0423 - val_accuracy: 0.9865
Epoch 4/300
8463/8463 [==============================] - 261s 31ms/step - loss: 0.0227 - accuracy: 0.9938 - val_loss: 2.8194 - val_accuracy: 0.6819
Epoch 5/300
8463/8463 [==============================] - 259s 31ms/step - loss: 0.0074 - accuracy: 0.9982 - val_loss: 0.0086 - val_accuracy: 0.9975
Epoch 6/300
8463/8463 [==============================] - 259s 31ms/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 0.0015 - val_accuracy: 0.9997
Epoch 7/300
8463/8463 [==============================] - 258s 31ms/step - loss: 6.0939e-04 - accuracy: 0.9999 - 

KeyboardInterrupt: 

In [12]:
SAVE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier_Compact\Models\Keras SqueezeNet Model\h5 Model\squeezenet.h5"
sn.save(SAVE_PATH)

In [13]:
TFLITE_SAVE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier_Compact\Models\Keras SqueezeNet Model\tflite Model\model.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(sn)
tfmodel = converter.convert()
open(TFLITE_SAVE_PATH, 'wb').write(tfmodel)

INFO:tensorflow:Assets written to: C:\Users\offco\AppData\Local\Temp\tmpugbt5ce2\assets


INFO:tensorflow:Assets written to: C:\Users\offco\AppData\Local\Temp\tmpugbt5ce2\assets


2930736